# Langchain: The basics

#### Developed By: Manaranjan Pradhan
#### www.manaranjanp.com

*This Jupyter notebook is confidential and proprietary to Manaranjan Pradhan. It is intended solely for authorized training purposes. Unauthorized distribution, sharing, or reproduction of this notebook or its contents is strictly prohibited. This material is for personal learning within the training program only and may not be used for commercial purposes or shared with others. Unauthorized use may result in disciplinary action or legal consequences. If you have received this notebook without authorization, please contact manaranjan@gmail.com immediately and delete all copies.*

In [1]:
!pip -q install openai langchain langchain_openai langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 8.2 MB/s eta 0:00:00


In [2]:
import os
from getpass import getpass

In [3]:
#os.environ['OPENAI_API_KEY'] = getpass('OPENAI_API_KEY')
os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


## Zero Shot Prompting



In [13]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain.chains import LLMChain

In [5]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=256,
    max_retries=2,
)

In [6]:
text = """What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Sentiment:"""

print(llm.invoke(text))

content='Sentiment: Positive. The customer uses words like "fresh", "flavorful", and "loved" to describe their experience, indicating a positive sentiment.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 80, 'total_tokens': 114, 'completion_time': 0.123636364, 'prompt_time': 0.003749288, 'queue_time': 0.199219914, 'total_time': 0.127385652}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'finish_reason': 'stop', 'logprobs': None} id='run--8c4ccec1-9a50-4a94-9079-c217071a957a-0' usage_metadata={'input_tokens': 80, 'output_tokens': 34, 'total_tokens': 114}


## Classifying List of Reviews

In [46]:
reviews = [
    "The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.",
    "Impressive service! The staff was attentive and made excellent recommendations. Thoroughly enjoyed the evening.",
    "The restaurant's interior was a visual treat, beautifully paired with their gourmet dishes. A delightful experience!",
    "The food tasted alright, but the tables were not very clean, which was off-putting.",
    "Waited 30 minutes even with a reservation, and the main course was served cold. Disappointing visit."
]

## Using Prompt Templates

In [47]:
from langchain import PromptTemplate

In [48]:
sentiment_template = """
What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: {review_text}
Sentiment:
"""

review_prompt = PromptTemplate(
    input_variables=["review_text"],
    template=sentiment_template,
)

### Generating Prompt with Templates

In [49]:
print(review_prompt.format(review_text=reviews[0]))


What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Sentiment:



### Calling LLM with the prompt template

In [34]:
sentiment_chain = review_prompt | llm

In [35]:
response = sentiment_chain.invoke({"review_text": reviews[0]})

response.content

'Sentiment: Positive'

In [36]:
print("================================")

for review in reviews:
    print(review_prompt.format(review_text=review))
    response = sentiment_chain.invoke({"review_text": review})
    print(response.content)
    print("================================")


What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Sentiment:

Sentiment: Positive

What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: Impressive service! The staff was attentive and made excellent recommendations. Thoroughly enjoyed the evening.
Sentiment:

Sentiment: Positive

What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The restaurant's interior was a visual treat, beautifully paired with their gourmet dishes. A delightful experience!
Sentiment:

Sentiment: Positive

What is the sentiment of the customer review given below? It should be a positive or negative sentiment.

review: The food tasted alright, but the tables were not very clean, which was off-putting.
Sentiment:

Sentiment: Negative

A

## Classifying Categories

In [38]:
category_template = """
Classify the reviews into one the four categories as given in the examples.

review: The grilled chicken was seasoned to perfection and simply melted in the mouth.
category: Food Quality

review: Despite the crowd, the place was immaculately clean and the restrooms were spotless.
category: Overall Hygiene

review: The dim lighting and soothing jazz music provided an intimate and romantic setting.
category: Restaurant Ambience

review: We were kept waiting for our table even after a confirmed reservation and the staff seemed disinterested.
category: Customer Service

review: {review_text}
Sentiment:
"""

category_prompt = PromptTemplate(
    input_variables=["review_text"],
    template=category_template,
)

In [39]:
# We can now generate a prompt using the `format` method.
print(category_prompt.format(review_text=reviews[0]))


Classify the reviews into one the four categories as given in the examples.

review: The grilled chicken was seasoned to perfection and simply melted in the mouth.
category: Food Quality

review: Despite the crowd, the place was immaculately clean and the restrooms were spotless.
category: Overall Hygiene

review: The dim lighting and soothing jazz music provided an intimate and romantic setting.
category: Restaurant Ambience

review: We were kept waiting for our table even after a confirmed reservation and the staff seemed disinterested.
category: Customer Service

review: The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Sentiment:



In [43]:
category_chain = category_prompt | llm

In [45]:
# Run the chain only specifying the input variable.
response = category_chain.invoke({"review_text": reviews[0]})

response.content

'The review mentions both the food ("The seafood platter was fresh and flavorful") and the ambiance ("Loved the chic décor and the ambiance of the place"), but since it touches on two categories, the most prominent one seems to be the overall appreciation of the place which includes food and ambiance. However, given the options and the fact that it specifically talks about "chic décor and the ambiance," it leans more towards:\n\ncategory: Restaurant Ambience'

In [ ]:
for review in reviews:
    print(review)
    print(category_chain.run({"review_text": review}))
    print("================================")

The seafood platter was fresh and flavorful. Loved the chic décor and the ambiance of the place.
Based on the given examples, this review can be classified into two categories, but since it has to be classified into one, I would choose: 

Category: Food Quality 

Reason: Although the review mentions the ambiance, the primary focus is on the food ("The seafood platter was fresh and flavorful").
Impressive service! The staff was attentive and made excellent recommendations. Thoroughly enjoyed the evening.
Category: Customer Service
The restaurant's interior was a visual treat, beautifully paired with their gourmet dishes. A delightful experience!
Based on the given examples, I would categorize the review as:

Category: Restaurant Ambience

The review mentions the restaurant's interior and the visual appeal, which aligns with the "Restaurant Ambience" category. Although it also mentions the gourmet dishes, the primary focus is on the ambiance and the overall experience, rather than just t

## Chaining Multiple Prompts

In [54]:
response_to_customer = """
"Write an appropriate and concise response to the customer by either appreciating their positive experience or adressing the specific concern that customer might
have mentioned in the feedback below.

<feedback>
{feedback}
</feedback>

The feedback is about {category}.

The goal of the response is to ensure customer engagement.
"""

In [55]:
response_prompt = PromptTemplate(
    input_variables=["feedback", "category"],
    template=response_to_customer,
)

In [56]:
final_response_chain = response_prompt | llm

In [57]:
complete_chain = (
    {
        "feedback": sentiment_chain,
        "category": category_chain,
    }
    | final_response_chain
)

In [58]:
from pprint import pprint

In [59]:
reviews[3]

'The food tasted alright, but the tables were not very clean, which was off-putting.'

In [62]:
response = complete_chain.invoke({"review_text": reviews[4]})

In [63]:
pprint(response.content)

('I apologize for the disappointing experience you had at our restaurant. We '
 'fell short of our standards by not seating you promptly with a confirmed '
 'reservation and serving your main course cold. I want to assure you that we '
 'take these issues seriously and will address them with our staff to prevent '
 'such incidents in the future. Can you please contact us directly so we can '
 'make things right and provide a better experience for you? Your feedback is '
 'invaluable in helping us improve our customer service.')
